Finally, let's look at how to work with regions of interest, or ROIs. These can be a bit difficult to deal with using the default OMERO API, so we have engineered ezomero to alleviate some of these issues.

I will start by recommending that you create an ROI or two in one of your images using the OMERO.web interface - you can open one of your images with OMERO.iviewer (right click the image name on the left-hand column, choose "open with", "omero.iviewer"), then select the "ROI" tab on the right-hand side and pick one of the drawing tools. Don't forget to save after drawing them.
Also, before we start, a small explanation on the OMERO format for ROIs: there are two separate objects in the OME data format named _ROI_ and _Shape_. A single ROI can contain multiple Shapes. In our examples, we will largely keep to ROIs that have a single shape, but the reason this distinction exists in code is to accommodate all cases that can have multiple shapes!

Let's start with the usual: imports and creating a connection.

In [ ]:
from config import OMEROUSER, OMEROPASS, OMEROHOST, OMEROPORT
import ezomero

conn = ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True)

Now, let's pick an image ID that contains ROIs and list the ROI IDs associated to that image.

In [ ]:
image_id = YOUR_IMAGE_ID_HERE
roi_ids = ezomero.get_roi_ids(conn, image_id)
print(roi_ids)

Now, because a single ROI can have multiple Shapes, we need to get the _Shape_ IDs belonging to that ROI:

In [ ]:
shape_ids = ezomero.get_shape_ids(conn, roi_ids[0])
print(shape_ids)

Finally, we will retrieve the actual Shape object from the first shape in the first ROI of that image:

In [ ]:
return_val = ezomero.get_shape(conn, shape_ids[0])
print(return_val)

You will see that this returs a tuple with an ezomero Shape object, plus color and stroke information. The Shape object will be of a type corresponding to the shape in the ROI you drew. The specific data you will get will depend on which kind of ROI you drew (a Rectangle will have x, y, width and height, for example). 

If you are not interested in stroke/color information, you can get a simpler result by doing

In [ ]:
shape, _, _, _ = ezomero.get_shape(conn, shape_ids[0])
print(shape)

With those two ways of doing this, you can get e.g. the X coordinate of your shape by doing

In [ ]:
print(shape.x)
print(return_val[0].x)

either one will get you the same result!

Now we can try to do the opposite: create a Shape object and pass it via ezomero to the server, adding it to the image. Let's try an Ellipse.

In [ ]:
from ezomero.rois import Ellipse
myshape = Ellipse(30, 40, 10, 20, label="my new ellipse")

Note that `post_roi` wants a LIST of shapes, so if you only have one you need to make a list out of it!

In [ ]:
new_roi_id = ezomero.post_roi(conn, image_id, [myshape])

Make sure to refresh your OMERO.web view and you should have a fresh new Ellipse ROI added to your image!

Don't forget to close your connection.

In [ ]:
conn.close()